In [1]:
import pandas as pd
import ast
import re

In [2]:
df = pd.read_csv("data/super_scotus_raw.csv")

In [3]:
df

,Unnamed: 0,id,year,citation,title,petitioner,respondent,docket_no,court,decided_date,...,votes_detail,is_eq_divided,votes_side,scdb_elements,convos,oyez_summary,justia_summary,wikipedia_summary,justia_sections,companion_cases
0,0,2010_09-479,2010,562 US 8,Abbott v. United States,Kevin Abbott,United States,09-479,Roberts Court,"Nov 15, 2010",...,"{'j__john_g_roberts_jr': 1.0, 'j__elena_kagan'...",False,"{'j__john_g_roberts_jr': 0.0, 'j__antonin_scal...","{'caseId': '2010-002', 'docketId': '2010-002-0...","{'ids': ['22372'], 'speaker': {'david_l_horan'...","{'Facts of the case': ['Facts of the case', ""I...","{'opinion_summary': [], 'primary_holding': []}",Mandatory minimum sentences under federal sent...,"{'Syllabus': '\n\n\nSYLLABUS\nOCTOBER TERM, 20...",False
1,1,2010_10-174,2010,564 US _,"American Electric Power Co., Inc. v. Connecticut","American Electric Power Company Inc., et al.","Connecticut, et al.",10-174,Roberts Court,"Jun 20, 2011",...,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...",False,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...","{'caseId': '2010-072', 'docketId': '2010-072-0...","{'ids': ['21694'], 'speaker': {'peter_d_keisle...","{'Facts of the case': ['Facts of the case', 'E...","{'opinion_summary': ['Plaintiffs, several stat...",[],"{'Syllabus': '\n\n\nSYLLABUS\nOCTOBER TERM, 20...",False
2,2,2010_09-987,2010,563 US 125,Arizona Christian School Tuition Organization ...,Arizona Christian School Tuition Organization,"Kathleen M. Winn, et al.",09-987,Roberts Court,"Apr 4, 2011",...,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...",False,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...","{'caseId': '2010-034', 'docketId': '2010-034-0...","{'ids': ['22462'], 'speaker': {'neal_kumar_kat...","{'Facts of the case': ['Facts of the case', ""A...","{'opinion_summary': [], 'primary_holding': []}",[],"{'Syllabus': '\n\n\nSYLLABUS\nOCTOBER TERM, 20...",False
3,3,2010_10-238,2010,564 US _,Arizona Free Enterprise Club's Freedom Club PA...,Arizona Free Enterprise Club's Freedom Club PA...,"Ken Bennett, in His Official Capacity as Arizo...",10-238,Roberts Court,"Jun 27, 2011",...,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...",False,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...","{'caseId': '2010-084', 'docketId': '2010-084-0...","{'ids': ['21834'], 'speaker': {'william_r_maur...","{'Facts of the case': ['Facts of the case', 'A...",{'opinion_summary': ['The Arizona Citizens Cle...,[],"{'Syllabus': ""\n\n\nSYLLABUS\nOCTOBER TERM, 20...",False
4,4,2010_10-98,2010,563 US 731,Ashcroft v. Al-Kidd,John Ashcroft,Abdullah al-Kidd,10-98,Roberts Court,"May 31, 2011",...,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...",False,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...","{'caseId': '2010-052', 'docketId': '2010-052-0...","{'ids': ['23016'], 'speaker': {'neal_kumar_kat...","{'Facts of the case': ['Facts of the case', 'I...","{'opinion_summary': [""Respondent alleged that,...",[],"{'Syllabus': '\n\n\nSYLLABUS\nOCTOBER TERM, 20...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,401,2015_15-6418,2015,578 US _,Welch v. United States,Gregory Welch,United States,15-6418,Roberts Court,"Apr 18, 2016",...,"{'j__john_g_roberts_jr': 1.0, 'j__anthony_m_ke...",False,"{'j__john_g_roberts_jr': 1.0, 'j__anthony_m_ke...","{'caseId': '2015-055', 'docketId': '2015-055-0...","{'ids': ['24114'], 'speaker': {'amir_h_ali': {...","{'Facts of the case': ['Facts of the case', 'P...","{'opinion_summary': [""Possession of a firearm ...",[],{'Syllabus': '\n\nNOTE:\u2002Where it is feasi...,False
402,402,2015_15-274,2015,579 US _,Whole Woman’s Health v. Hellerstedt,"Whole Woman’s Health, et al.","John Hellerstedt, Commissioner, Texas Departme...",15-274,Roberts Court,"Jun 27, 2016",...,"{'j__john_g_roberts_jr': 2.0, 'j__anthony_m_ke...",False,"{'j__john_g_roberts_jr': 0.0, 'j__anthony_m_ke...","{'caseId': '2015-048', 'docketId': '2015-048-0...","{'ids': ['24108'], 'speaker': {'stepha

In [4]:
df.columns

Index(['Unnamed: 0', 'id', 'year', 'citation', 'title', 'petitioner',
       'respondent', 'docket_no', 'court', 'decided_date', 'url',
       'transcripts', 'adv_sides_inferred', 'known_respondent_adv',
       'advocates', 'win_side', 'win_side_detail', 'scdb_docket_id', 'votes',
       'votes_detail', 'is_eq_divided', 'votes_side', 'scdb_elements',
       'convos', 'oyez_summary', 'justia_summary', 'wikipedia_summary',
       'justia_sections', 'companion_cases'],
      dtype='object')

## Convert dictionary-like strings to dictionaries

In [5]:
def parse_dictionary_value(value):
    # Replace NaN values with None
    value = re.sub(r'nan', 'None', str(value))
    # Remove unnecessary characters using regex
    value = re.sub(r'[^\x00-\x7F]+', '', value)
    # Remove words starting with "\"
    value = re.sub(r'\b\\[a-zA-Z0-9_]+\b', '', value)
    try:
        # Try to parse the string as a dictionary
        parsed_dict = ast.literal_eval(value)
        return parsed_dict
    except (SyntaxError, ValueError):
        return None

In [6]:
# List of columns containing dictionary-like strings
str_to_dict_cols = ["votes_detail", "scdb_elements", "convos", "oyez_summary", "justia_summary"]

for col in str_to_dict_cols:
    df[col] = df[col].fillna('{}')  # Fill null values with an empty dictionary
    df[col] = df[col].apply(parse_dictionary_value)

In [7]:
# Define delimiters for different sections
opinion_marker = "Opinion"
concurrence_marker = "Concurrence"
dissent_marker = "Dissent"

# Function to separate sections for a given text string
def separate_sections(text_string):
    # Find the start and end positions of each section
    opinion_start = text_string.find(opinion_marker)
    concurrence_start = text_string.find(concurrence_marker)
    dissent_start = text_string.find(dissent_marker)

    # Extract sections
    opinion_section = text_string[opinion_start + len(opinion_marker):concurrence_start].strip()
    concurrence_section = text_string[concurrence_start + len(concurrence_marker):dissent_start].strip()
    dissent_section = text_string[dissent_start + len(dissent_marker):].strip()

    # Store sections in a dictionary
    sections_dict = {"Opinion": opinion_section}

    # Check if concurrence and dissent sections exist and add them to the dictionary
    if concurrence_start != -1:
        sections_dict["Concurrence"] = concurrence_section
    if dissent_start != -1:
        sections_dict["Dissent"] = dissent_section

    return sections_dict

In [8]:
df["justia_sections"] = df["justia_sections"].apply(separate_sections)

In [9]:
# Convert float to int
df["win_side"] = df["win_side"].astype(int)

In [10]:
# Check if all of the columns that should be a dict, are indeed a dict
## Omitted the "convos" column as it didn't pass and its not needed for this project
test_cases = {
    "votes_detail": {"all_dicts": True},
    "scdb_elements": {"all_dicts": True},
    "oyez_summary": {"all_dicts": True},
    "justia_summary": {"all_dicts": True},
    "justia_sections": {"all_dicts": True}
}

for col, expected_result in test_cases.items():
    non_dict_rows = []
    for idx, value in df[col].items():
        if not isinstance(value, dict):
            non_dict_rows.append(idx)
    assert len(non_dict_rows) == 0, f"Test case failed: {col} contains non-dictionary values"
    print(f"Test case passed: {col} contains only dictionary values")

Test case passed: votes_detail contains only dictionary values
Test case passed: scdb_elements contains only dictionary values
Test case passed: oyez_summary contains only dictionary values
Test case passed: justia_summary contains only dictionary values
Test case passed: justia_sections contains only dictionary values


In [11]:
assert set().union(*(d.keys() for d in df["justia_sections"])) == {'Concurrence', 'Dissent', 'Opinion'}

## Subset data

In [12]:
df.iloc[0]

Unnamed: 0                                                              0
id                                                            2010_09-479
year                                                                 2010
citation                                                         562 US 8
title                                             Abbott v. United States
petitioner                                                   Kevin Abbott
respondent                                                  United States
docket_no                                                          09-479
court                                                       Roberts Court
decided_date                                                 Nov 15, 2010
url                                https://www.oyez.org/cases/2010/09-479
transcripts             [{'name': 'Oral Argument - October 04, 2010', ...
adv_sides_inferred                                                  False
known_respondent_adv                  

In [13]:
df = df[['votes_detail', 'win_side','title', 'petitioner', 'respondent', 'year', 'docket_no', 'justia_summary', 'justia_sections']]

In [14]:
df

,votes_detail,win_side,title,petitioner,respondent,year,docket_no,justia_summary,justia_sections
0,"{'j__john_g_roberts_jr': 1.0, 'j__elena_kagan'...",0,Abbott v. United States,Kevin Abbott,United States,2010,09-479,"{'opinion_summary': [], 'primary_holding': []}",{'Opinion': '(Ginsburg)': '\n\n\nOPINION OF TH...
1,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...",1,"American Electric Power Co., Inc. v. Connecticut","American Electric Power Company Inc., et al.","Connecticut, et al.",2010,10-174,"{'opinion_summary': ['Plaintiffs, several stat...",{'Opinion': '(Ginsburg)': '\n\n\nOPINION OF TH...
2,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...",1,Arizona Christian School Tuition Organization ...,Arizona Christian School Tuition Organization,"Kathleen M. Winn, et al.",2010,09-987,"{'opinion_summary': [], 'primary_holding': []}",{'Opinion': '(Kennedy)': '\n\n\nOPINION OF THE...
3,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...",1,Arizona Free Enterprise Club's Freedom Club PA...,Arizona Free Enterprise Club's Freedom Club PA...,"Ken Bennett, in His Official Capacity as Arizo...",2010,10-238,{'opinion_summary': ['The Arizona Citizens Cle...,"{'Opinion': '(Roberts)': ""\n\n\nOPINION OF THE..."
4,"{'j__john_g_roberts_jr': 1.0, 'j__antonin_scal...",1,Ashcroft v. Al-Kidd,John Ashcroft,Abdullah al-Kidd,2010,10-98,"{'opinion_summary': ['Respondent alleged that,...",{'Opinion': '(Scalia)': '\n\n\nOPINION OF THE ...
...,...,...,...,...,...,...,...,...,...
401,"{'j__john_g_roberts_jr': 1.0, 'j__anthony_m_ke...",1,Welch v. United States,Gregory Welch,United States,2015,15-6418,{'opinion_summary': ['Possession of a firearm ...,{'Opinion': '(Kennedy)': '\n\nNOTICE:\u2002Thi...
402,"{'j__john_g_roberts_jr': 2.0, 'j__anthony_m_ke...",1,Whole Woman’s Health v. Hellerstedt,"Whole Woman’s Health, et al.","John Hellerstedt, Commissioner, Texas Departme...",2015,15-274,{'opinion_summary': ['Texas House Bill 2 (2013...,{'Opinion': '(Breyer)': '\n\nNOTICE:\u2002This...
403,"{'j__john_g_roberts_jr': 2.0, 'j__anthony_m_ke...",1,Williams v. Pennsylvania,Terrance Williams,Pennsylvania,2015,15-5040,{'opinion_summary': ['Williams was convicted o...,{'Opinion': '(Kennedy)': '\n\nNOTICE:\u2002Thi...
404,"{'j__john_g_roberts_jr': 1.0, 'j__anthony_m_ke...",0,Wittman v. Personhuballah,"Robert J. Wittman, et al.","Gloria Personhuballah, et al.",2015,14-1504,{'opinion_summary': ['Voters from Virginias Co...,{'Opinion': '(Breyer)': '\n\nNOTICE:\u2002This...


## Export data

In [15]:
df.to_csv("data/super_scotus_cleaned.csv", index = False)